In [3]:
package DroneModelTdSE2024 {

    
    package ForestFireObservationDrone {

        import Drone_SharedAssetsSuperset::**;

        part forestFireObservationDrone : Drone {

            // Variant selections

            // numberOfEngines
            :>> numberOfEnginesVariation = numberOfEnginesVariation::sixEngines;
            :>> body = DroneBodyVariation::droneBody6Engines;
            :>> flightControl = FlightControlVariation::droneFlightControl6Engines;

            // Battery
            :>> battery = DroneBatteryVariation::longDistanceBattery;       

            // Engine
            part engine1 :> engines {
                :>> actualMass = 100 [SI::g];
                :>> propKind = PropKind::clockwise;
            }
            part engine2 :> engines {
                :>> actualMass = 100 [SI::g];                
                :>> propKind = PropKind::counterclockwise;
            }
            part engine3 :> engines {
                :>> actualMass = 100 [SI::g];
                :>> propKind = PropKind::clockwise;
            }        
            part engine4 :> engines {
                :>> actualMass = 100 [SI::g];                
                :>> propKind = PropKind::counterclockwise;
            }
            part engine5 :> engines {
                :>> actualMass = 100 [SI::g];                
                :>> propKind = PropKind::clockwise;
            }
            part engine6 :> engines {
                :>> actualMass = 100 [SI::g];                
                :>> propKind = PropKind::counterclockwise;
            }
        }
        
        // Requirement satisfaction
        satisfy Drone_Requirements::REQ1 by forestFireObservationDrone;
        satisfy Drone_Requirements::REQ6 by forestFireObservationDrone;
        
    }
    
    
    library package Drone_SharedAssetsSuperset {

        package Drone {

            import DroneBattery::**;
            import DroneEngine::**;
            import DroneBody::**;
            import DroneFlightControl::**;  

            part def Drone {

                // Attributes
                attribute actualMass : ISQ::MassValue = battery.actualMass + body.actualMass + QuantityCalculations::sum(engines.actualMass);

                // Spatial extent (simplification drone boundingshape = body boundingbox)
                item boundingBox : ShapeItems::Box [1] :> boundingShapes = body.boundingBox;

                // Power
                part battery : DroneBatteryVariation;
                message notifyBatteryLow of SigBatteryLow from battery.batteryManagementSystem to flightControl;

                // Engines
                variation attribute numberOfEnginesVariation : ScalarValues::Integer {
                    variant attribute fourEngines = 4;
                    variant attribute sixEngines = 6;
                }
                part engines[numberOfEnginesVariation] : DroneEngine;

                // Body
                part body : DroneBodyVariation;

                // Flight Control System
                part flightControl : FlightControlVariation;

                // Connections
                connect battery to flightControl;
                connect battery to body;                
            }
            
            package Drone_Requirements {

                requirement <REQ1> height {
                    doc /*
                        * The drone shall not be higher than the maximumHeight of 60 cm.
                        */
                    subject d : Drone;
                    
                    attribute maximumHeight : ISQBase::LengthValue = 60 [SI::cm];
                    
                    require constraint {
                        d.boundingBox.height <= maximumHeight
                    }
                }
                requirement <REQ6> lightweight {
                    doc /*
                        * The drone shall not weigh more than maximumMass of 8 kg.
                        */
                    subject d : Drone;
                    
                    attribute maximumMass : ISQBase::MassValue = 8 [SI::kg];
                    
                    require constraint {
                        d.actualMass <= maximumMass
                    }
                        
                }
            }            
        }

        //
        // Drone Engine Shared Assets
        //
        package DroneEngine {
            package DroneEngine_Parts {
                part def DroneEngine {
                    // Attributes
                    attribute actualMass : ISQBase::MassValue; 
                    
                    enum propKind : PropKind;
                    enum def PropKind {
                        enum clockwise;
                        enum counterclockwise;
                    }            
                }
            }                        
        }

        //
        // Drone Body Shared Assets
        //
        package DroneBody {

            package DroneBody_Parts {

                import DroneBody_Domain::*;

                variation part def DroneBodyVariation :> DroneBody {
                    variant part droneBody4Engines : DroneBody {
                        attribute :>> actualMass = 0 [SI::g];
                        part :>> rotors[4];
                        part :>> arms[2];
                    }
                    variant part droneBody6Engines : DroneBody {
                        attribute :>> actualMass = 0 [SI::g];
                        part :>> rotors[6];
                        part :>> arms[3];
                    }
                }            

                abstract part def DroneBody {
                    
                    // Attributes
                    attribute actualMass : ISQBase::MassValue; 
                                        
                    // Spatial extent
                    item :>> shape : ShapeItems::Cylinder {
                        attribute :>> height = 0 [SI::cm];
                        item :>> base.shape : ShapeItems::Circle {
                             attribute :>> radius = 0 [SI::cm];
                        }
                    }
                    item boundingBox : ShapeItems::Box [1] :> boundingShapes {
                        attribute :>> length = 0 [SI::cm];
                        attribute :>> width  = 0 [SI::cm];
                        attribute :>> height = 0 [SI::cm];
                    }            
                    part rotors[*];
                    part arms[*];
                    connect rotors to arms;
                    part landingGear;
                    part coreBody;
                }
            }

            package DroneBody_Domain {            
                enum def BodyColorKind {
                    enum red;
                    enum white;
                    enum blue;
                    enum black;
                }
            }
        }


        //
        // Drone Flight Control Software Shared Assets
        //
        package DroneFlightControl {

            variation part def FlightControlVariation :> DroneFlightControl {
                variant part droneFlightControl4Engines : DroneFlightControl;
                variant part droneFlightControl6Engines : DroneFlightControl;
            }
            part def DroneFlightControl;
        }

        //
        // Drone Battery Shared Assets
        //     
        package DroneBattery {

            package DroneBattery_Domain {
                attribute def SigBatteryLow {
                    attribute capacity : ISQElectromagnetism::ElectricChargeValue;
                }
            }

            package DroneBattery_Parts {

                variation part def DroneBatteryVariation :> DroneBattery {
                    variant part standardBattery : DroneBattery  {
                        attribute :>> actualMass = 0 [SI::g];
                        attribute :>> maxCapacity = 13680 [SI::'C'];
                    }
                    variant part longDistanceBattery : DroneBattery {
                        attribute :>> actualMass = 0 [SI::g];
                        attribute :>> maxCapacity = 18000 [SI::'C'];
                    }
                }            

                abstract part def DroneBattery {                    
                    // Attributes
                    attribute actualMass : ISQBase::MassValue; 
                    attribute maxCapacity : ISQElectromagnetism::ElectricChargeValue;
                    enum currentCapacityLevel : BatteryCapacityKind;
                    enum def BatteryCapacityKind :> ScalarValues::Integer {
                        enum full = 100;
                        enum medium = 50;
                        enum low = 30;
                        enum critical = 15;
                    }

                    // Parts
                    part batteryManagementSystem {
                        doc /* The Battery Management System (BMS) is a control unit that manages the charging and 
                             * discharging of the battery cells. It ensures that the cells are charged and discharged 
                             * correctly and safely.
                             */
                    }
                }
            }

        }    
    }
}




Package DroneModelTdSE2024 (a49d88bc-9f7f-4c0f-b84d-c44885941c60)


In [4]:
%publish DroneModelTdSE2024

API base path: http://sysml2.intercax.com:9000
Processing..........
Posting Commit (1024 elements)...305b299a-523b-4703-90b1-5b5763791ee8



Saved to Project DroneModelTdSE2024 Tue May 28 21:02:37 CEST 2024 (3eafe69f-4610-4403-861f-09cf217c1696)


In [13]:

  part def SEM {
    part instrumentControl : InstrumentControl {
      port electronGunInterface : ElectronGunInterface;
      port electronBeamControlInterface : ElectronBeamControlInterface;
    }
    part systemIntegration : SystemIntegration {
      port electronGunInterface : ~ElectronGunInterface;
      port electronBeamControlInterface : ~ElectronBeamControlInterface;

    }

    connect instrumentControl.electronGunInterface to systemIntegration.electronGunInterface;
    connect instrumentControl.electronBeamControlInterface to systemIntegration.electronBeamControlInterface;
  }

  port def ElectronGunInterface {
    out item electronBeamOutput;
    out item highVoltageControl;
    out item filamentControl;
  }

  port def ElectronBeamControlInterface {
    out item beamDeflectionControl;
    out item scanSignals;
    out item beamBlankerControl;
  }

  


PartDefinition SEM (787d22c4-2566-433c-8dd2-e20dfb5d8b43)
PortDefinition ElectronGunInterface (94951780-4f0d-4273-82d5-7793aef9b6e8)
PortDefinition ElectronBeamControlInterface (eff8e981-e16f-45e6-9263-8fb3b38b81a6)
